### Sum 

In [1]:
import numpy as np
import tvm
from tvm import te

In [2]:
a = np.random.normal(size=(3,4)).astype('float32')
a.sum(axis=1)

array([ 1.3237908 , -1.2010362 ,  0.60389924], dtype=float32)

In [3]:
def sum_rows(a,b):
    n = len(b)
    for i in range(n):
        b[i] = np.sum(a[i,:])
        
b = np.empty((3,),dtype='float32')
sum_rows(a, b)
b

array([ 1.3237908 , -1.2010362 ,  0.60389924], dtype=float32)

In [4]:
n,m = te.var('n'), te.var('m')
A = te.placeholder((n,m),name='a')
j = te.reduce_axis((0,m),name='j')
B = te.compute((n,), lambda i:te.sum(A[i,j],axis=j),name='b')
s = te.create_schedule(B.op)
tvm.lower(s,[A,B],simple_mode=True)

produce b {
  for (i, 0, n) {
    b[(i*stride)] = 0f
    for (j, 0, m) {
      b[(i*stride)] = (b[(i*stride)] + a[((i*stride) + (j*stride))])
    }
  }
}

In [5]:
mod = tvm.build(s,[A,B])
c = tvm.nd.array(np.empty((3,),dtype='float32'))
mod(tvm.nd.array(a),c)
np.testing.assert_equal(b, c.asnumpy())

In [9]:
### reduce to 0-rank tensor
i = te.reduce_axis((0, n), name='i')
B = te.compute((), lambda: te.sum(A[i, j], axis=(i, j)), name='b')
s = te.create_schedule(B.op)
tvm.lower(s, [A, B], simple_mode=True)

mod = tvm.build(s,[A,B])
c = tvm.nd.array(np.empty((), dtype='float32'))
mod(tvm.nd.array(a),c)
np.testing.assert_allclose(a.sum(),c.asnumpy(),atol=1e-5)